In [1]:
import torch
from torch.optim import lr_scheduler
import pandas as pd

In [2]:
import os, sys
sys.path.insert(0, os.path.realpath(os.path.pardir))

from utils.config import cfg
from utils.blocks.forecaster import Forecaster
from utils.blocks.encoder import Encoder
from utils.tools.ordered_easydict import OrderedDict
from utils.blocks.module import EF
from utils.loss import Weighted_mse_mae
from utils.train_and_test import train_and_test
from utils.net_params import conv2d_params
from utils.blocks.module import Predictor

In [3]:
batch_size = cfg.GLOBAL.BATCH_SIZE
max_iterations = 10 #80000
test_iteration_interval = 5 #10000
test_and_save_checkpoint_iterations = 5 #10000

LR = 1e-4

criterion = Weighted_mse_mae().to(cfg.GLOBAL.DEVICE)

model = Predictor(conv2d_params).to(cfg.GLOBAL.DEVICE)
# data = torch.randn(5, 4, 1, 2034, 2048)
# output = model(data)
# print(output.size())

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-6)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20000, gamma=0.7)
folder_name = "convLSTM"

In [4]:
data = torch.randn(5, 4, 1, 2034, 2048)
output = model(data)
print(output.size())

c:\Users\FACT-PC\anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


torch.Size([20, 4, 1, 2034, 2048])


In [5]:
model

Predictor(
  (model): Sequential(
    (conv1_relu_1): Conv2d(5, 64, kernel_size=(7, 7), stride=(5, 5), padding=(7, 1))
    (relu_conv1_relu_1): ReLU(inplace=True)
    (conv2_relu_1): Conv2d(64, 192, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (relu_conv2_relu_1): ReLU(inplace=True)
    (conv3_relu_1): Conv2d(192, 576, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (relu_conv3_relu_1): ReLU(inplace=True)
    (conv4_relu_1): Conv2d(576, 1728, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu_conv4_relu_1): ReLU(inplace=True)
    (conv5_relu_1): Conv2d(1728, 1728, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu_conv5_relu_1): ReLU(inplace=True)
    (deconv1_relu_1): ConvTranspose2d(1728, 1728, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (relu_deconv1_relu_1): ReLU(inplace=True)
    (deconv2_relu_1): ConvTranspose2d(1728, 576, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (relu_deconv2_relu_1): ReLU(inplace=True)
    (deconv3_r

In [ ]:
train_and_test(model, optimizer, criterion, exp_lr_scheduler, batch_size, max_iterations, test_iteration_interval, test_and_save_checkpoint_iterations, folder_name)